In [ ]:
import os
import imp
import numpy as np
import theano
import lasagne
import loading
from training import *
from network import *
from architectures import *

L = lasagne.layers
nl = lasagne.nonlinearities
T = theano.tensor
headdir = os.path.expanduser('~/Google Drive/Bas Zahy Gianni - Games')
datafile = os.path.join(headdir, 'Data/0_hvh/Clean/_summaries/model_input_with_groups.csv')
resultsdir = os.path.join(headdir, 'Analysis/0_hvh/Loglik/nns')

In [ ]:
def default_convnet(input_var=None):
    input_shape = (None, 2, 4, 9)
    FixLayer = make_FixLayer(input_var)
    
    input_layer = L.InputLayer(shape=input_shape, input_var=input_var)
    network = L.Conv2DLayer(
        input_layer, num_filters=32, filter_size=(4,4), pad='full',
        nonlinearity=nl.very_leaky_rectify, W=lasagne.init.HeUniform(gain='relu') #GlorotUniform()
    )
    
    network = L.FeaturePoolLayer(network, pool_function=T.sum, pool_size=2)
    network = L.DropoutLayer(network, p=.75)
    network = L.DenseLayer(
        network, num_units=36, 
        nonlinearity=nl.very_leaky_rectify, W=lasagne.init.HeUniform(gain='relu')
    )
    network = L.NonlinearityLayer(network, nonlinearity=nl.softmax)
    network = FixLayer(network)
    
    return network

net = Network(default_convnet)

In [ ]:
data = loading.default_loader(datafile)
trainer = DefaultTrainer()
net_list = trainer.train_all(architecture=default_convnet, data=data, seed=985227)

In [ ]:
from scipy.stats import bayes_mvs as bmvs
bmvs([n.test_err for n in net_list], alpha=.95)

In [ ]:


D, groups, Xs, ys, Ss = loading.default_loader(datafile)

splitsize = len(Xs)
r = np.arange(splitsize)
r = np.tile(r, [splitsize, 1])
r = r + r.T
split_indices = r % splitsize

split = 0
train_i = split_indices[split, :3]
val_i = split_indices[split, 3:4]
test_i = split_indices[split, 4:]

X, y, S = [np.concatenate(np.array(Zs)[train_i]) for Zs in [Xs, ys, Ss]]
Xv, yv, Sv = [np.concatenate(np.array(Zs)[val_i]) for Zs in [Xs, ys, Ss]]
Xt, yt, St = [np.concatenate(np.array(Zs)[test_i]) for Zs in [Xs, ys, Ss]]
X, y = loading.augment((X, y))
S = np.concatenate([S, S, S, S])

In [ ]:
np.random.seed(985227)

net = Network(default_convnet)
trainer = Trainer()

trainer.train(net, training_data=(X, y), validation_data=(Xv, yv))
err, acc, bats = trainer.test(net, (Xt, yt))